# Verificar existencia de las empresas en Podio

## Files

In [ ]:
# Número de archivo a crear
nro_de_archivo = 3

# existendes
Path_Datos_Podio = 'Datos_Podio/iGTa _ StartUps y PyMEs - Última vista usada (2).xlsx' #ejemplo: 'Datos_Podio/iGTa _ StartUps y PyMEs - Última vista usada (2).xlsx'
Path_iGTa_N = 'iGTa_N_ok/iGTa_'+str(nro_de_archivo) +'.csv'

## Importo las librerías

In [ ]:
import pandas as pd
import numpy as np

## Funciones

In [ ]:
def limpiar_de_conectores(string):
    eliminar = [ '™', 'la ', 'el ', 'los ', 'lo ', 'the ', 'de ', 'y ', ',', '.', ';', ':', '(', ')', '-', '_', '!', '?', 'w/', '®', '.com']
    try:
        if string:
            string = string.lower()
            for item in eliminar:
                while string.find(item) != -1:
                    string = string.replace(item, '')
            return string
    except:
        print("error en limpiar_de_conectores()")

In [ ]:
def eliminar_espacios_no_utiles(string):
    try:
        if string:
            string = string.lower()
            while string[len(string)-1] == ' ':
                aux = list(string)
                aux.pop()
                string = "".join(aux)
            while string[0] == ' ':
                aux = list(string)
                aux.pop(0)
                string = "".join(aux)
            return string
    except:
        return string
        print("error en eliminar_espacios_no_utiles()")

In [ ]:
def df_add_dimension_by_spaces (df):
    for i in range(df.shape[0]):
        if df[i]:
            df[i] = str(df[i]).split(' ')
    return df

In [ ]:
# nuevo = str ; podio = df
# existe: 1=sí 0=no 2=revisar a mano None=no hay valor en nuevo
def comparar_nombres(nuevo, podio):
    try:    
        existe = 0
        coincidencias_max = 0
        coincidencias = 0
        for item_p in podio: 
            if item_p:
                if len(item_p) == 1:
                    if len(nuevo) == 1:
                        if item_p == nuevo:
                            return 1
                    elif len(nuevo) > 1:
                        for item_n in nuevo:
                            if item_n == item_p:
                                coincidencias = coincidencias + 1
                    else:
                        return None         
                elif len(item_p) > 1:
                    for item_p2 in item_p:
                        if len(nuevo) == 1:
                            if item_p2 == nuevo:
                                coincidencias = coincidencias + 1
                        elif len(nuevo) > 1:
                            for item_n in nuevo:
                                if item_n == item_p2:
                                    coincidencias = coincidencias + 1   
                if ((coincidencias/len(nuevo))>(2/3))or((coincidencias / len(item_p))>(2/3))or((coincidencias/len(nuevo))==1)or((coincidencias/len(item_p))==1):
                    return 1
                elif coincidencias > coincidencias_max:
                    coincidencias_max = coincidencias
                coincidencias = 0
        
        if coincidencias_max >= 2:
            return 2
        else:
            return 0 
    except:
        print("error en comparar_nombres()")

In [ ]:
def create_internal_df():
    df = {'Contacto_primer_nombre': [],
        'Contacto_nombre_completo': [],
        'Desc': [],
        'Mail': [],
        'Empresa1_nombre': [],
        'Empresa1_rol': [],
        'Empresa1_link': [],
        'Contacto_Linkedin': [],
        'Empresa_nombre': [],
        'Empresa_Linkedin': [],
        'existe': '',
       }
    df = pd.DataFrame (df, columns = ['Contacto_primer_nombre', 'Contacto_nombre_completo', 'Desc', 'Mail', 'Empresa1_nombre', 'Empresa1_rol', 'Empresa1_link','Contacto_Linkedin', 'Empresa_nombre','Empresa_Linkedin', 'existe'])
    return df

In [ ]:
def generate_csv_exists_n(df, dfn, N):
    if N == 0:
        for i in range(0, df.shape[0]):
            if pd.isna(df["existe"][i]) == False and pd.isna(df["Contacto_nombre_completo"][i]) == False and df["Empresa_nombre"][i] != '':
                if df["existe"][i] == np.float64(N):
                    dfn = dfn.append(df.loc[i], ignore_index = True)
            else:
                pass
                #print("NAN value in 'exist' data, row " +str(i)+" of the csv file")
    elif N == 1:
        for i in range(0, df.shape[0]):
            if pd.isna(df["existe"][i]) == False:
                if df["existe"][i] == np.float64(N):
                    dfn = dfn.append(df.loc[i], ignore_index = True)
            else:
                pass
                #print("NAN value in 'exist' data, row " +str(i)+" of the csv file")
    elif N == 2:
        for i in range(0, df.shape[0]):
            #if pd.isna(df["existe"][i]) == False:
            if df["existe"][i] == np.float64(N) or (df["existe"][i] != np.float64(0) and (pd.isna(df["Contacto_nombre_completo"][i]) == True or df["Empresa_nombre"][i] == '')):
                dfn = dfn.append(df.loc[i], ignore_index = True)
            else:
                pass
                #print("NAN value in 'exist' data, row " +str(i)+" of the csv file")
    return dfn

## Main program

In [ ]:
En_Podio = pd.read_excel(Path_Datos_Podio)["Nombre del organización"]
new_file = pd.read_csv(Path_iGTa_N)
for_check = new_file["Empresa_nombre"].copy()

for i in range(0, len(for_check)):
    for_check[i] = eliminar_espacios_no_utiles(limpiar_de_conectores(for_check[i]))
for i in range(0, len(En_Podio)):
    En_Podio[i] = eliminar_espacios_no_utiles(limpiar_de_conectores(En_Podio[i]))

En_Podio = df_add_dimension_by_spaces(En_Podio)
for_check = df_add_dimension_by_spaces(for_check)

existe = []
for item in for_check:
    existe.append(comparar_nombres(item, En_Podio))

new_file["existe"] = pd.DataFrame(existe, columns = ["existe"])
#new_file

In [ ]:
new_file_0 = create_internal_df()
new_file_1 = create_internal_df()
new_file_2 = create_internal_df()

new_file_0 = generate_csv_exists_n(new_file, new_file_0, 0)
new_file_1 = generate_csv_exists_n(new_file, new_file_1, 1)
new_file_2 = generate_csv_exists_n(new_file, new_file_2, 2)

In [ ]:
print(len(new_file_0),len(new_file_1), len(new_file_2))

In [ ]:
count = 0
for i in range(0, len(new_file_0)):
    if pd.isna(new_file_0["Contacto_nombre_completo"][i]) == True:
        count = count + 1
print("count of nulls in N0: " + str(count))

In [ ]:
new_file_0.to_csv("iGTa_N_E_ok/iGTa_N"+str(nro_de_archivo)+"_E0.csv")
new_file_1.to_csv("iGTa_N_E_ok/iGTa_N"+str(nro_de_archivo)+"_E1.csv")
new_file_2.to_csv("iGTa_N_E_ok/iGTa_N"+str(nro_de_archivo)+"_E2.csv")